# Tarea 2

### Cuerpo Docentes

- Profesores: [Andrés Abeliuk](https://aabeliuk.github.io/), [Fabián Villena](https://villena.cl/).
- Profesor Auxiliar: Martín Paredes

### Instrucciones generales

- Grupos de máximo 4 personas.
- Esta prohibido compartir las respuestas con otros grupos.
- Indicios de copia serán penalizados con la nota mínima.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente, si utiliza material extra debe citarlo.


### Integrantes

- Fabio Bórquez
- Luis Berríos
- Juan Román
- Marcos Molina

## Contexto

El discurso de odio es cualquier expresión que promueva o incite a la discriminación, la hostilidad o la violencia hacia una persona o grupo de personas en una relación asimétrica de poder, tal como la raza, la etnia, el género, la orientación sexual, la religión, la nacionalidad, una discapacidad u otra característica similar.

En cambio, la incivilidad se refiere a cualquier comportamiento o actitud que rompe las normas de respeto, cortesía y consideración en la interacción entre personas. Esta puede manifestarse de diversas formas, tal como insultos, ataques personales, sarcasmo, desprecio, entre otras.

En esta tarea tendrán a su disposición un dataset de textos con las etiquetas `odio`, `incivilidad` o `normal`. La mayor parte de los datos se encuentra en español de Chile. Con estos datos, deberán entrenar un modelo que sea capaz de predecir la etiqueta de un texto dado.

El corpus para esta tarea se compone de 3 datasets:  
- [Multilingual Resources for Offensive Language Detection de Arango et al. (2022)](https://aclanthology.org/2022.woah-1.pdf#page=136)
- [Dataton UTFSM No To Hate (2022)](http://dataton.inf.utfsm.cl/)
- Datos generados usando la [API de GPT3 (modelo DaVinci 03)](https://platform.openai.com/docs/models/gpt-3).

Agradecimientos a los autores por compartir los datos y a David Miranda, Fabián Diaz, Santiago Maass y Jorge Ortiz por revisar y reetiquetar los datos en el contexto del curso "Taller de Desarrollo de Proyectos de IA" (CC6409), Departamento de Ciencias de la Computación, Universidad de Chile.

Los datos solo pueden ser usados con fines de investigación y docencia. Está prohibida la difusión externa.


## Tarea a resolver

Para esta tarea 2, buscaremos desarrollar un *benchmark* sobre una tarea de clasificación de NLP. Un benchmark es básicamente utilizar diferentes técnicas para resolver una misma tarea específica, en este caso seguiremos buscando alternativas para resolver el problema de clasificación de la tarea 1. Particularmente, se le pide:

- Implementar una arquitectura en RNN utilizando PyTorch.
- Utilizar transformers para revolver el problema de clasificación, en especifico utilizar BETO.
- Utilizar algún LLM utilizando Zero y Few short learning para resolver el problema de clasificación.


### Cargar el dataset


En esta sección, cargaremos el dataset desde el repositorio del módulo. Para ello ejecute las siguientes líneas:

In [ ]:
import pandas as pd

In [ ]:
# Dataset.
dataset_df = pd.read_csv("https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/new/assignment_1/train/train.tsv", sep="\t")


### Analizar los datos

En esta sección analizaremos el balance de los datos. Para ello se imprime la cantidad de tweets de cada dataset agrupados por la intensidad de sentimiento.

In [ ]:
dataset_df.sample(5)

In [ ]:
dataset_df["clase"].value_counts()

In [ ]:
target_classes = list(dataset_df['clase'].unique())
target_classes

### Instalar librerias

Puede usar esta celda para instalar las librerías que estime necesario.

In [ ]:
%%capture



### Importar librerías

En esta sección, importamos la liberías necesarias para el correcto desarrollo de esta tarea. Puede utilizar otras librerías que no se en encuentran aquí, pero debe citar su fuente.

In [ ]:
!pip uninstall -y torch torchvision torchaudio

!pip install torch==2.3.0 --index-url https://download.pytorch.org/whl/cpu
!pip uninstall -y scipy
!pip install scipy==1.11.4
!pip install torchtext
!pip install scikit-plot

#falta este
!pip install gensim



In [ ]:
import nltk
import numpy as np

from nltk import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin

# importe aquí sus clasificadores

import matplotlib.pyplot as plt

# word2vec
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.phrases import Phrases, Phraser

# Pytorch imports
import torch
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

from torch.utils.data import DataLoader
from torchtext.data.functional import to_map_style_dataset

from torch import nn
from torch.nn import functional as F

from tqdm import tqdm
from sklearn.metrics import accuracy_score
import gc

from torch.optim import Adam
from torchtext.data import get_tokenizer



### Crear un clasificador basado en RNN

En esta parte de le pide definir un clasificador utilizando `PyTorch` con alguna arquitectura en Redes Recurrentes. Para ello debe realizar todos los pasos vistos en el tutorial 5, por lo que se recomienda revisarlo. Importante, no puede replicar ningún ejemplo de los del tutorial 5, debe proponer sus propias arquitecturas. Se le recomienda leer como utilizar variaciones de la RNN, como la LSTM o GRU en `Pytorch`.

Para completa esta parte deberá replicar el flujo de trabajo de como utilizar `PyTorch`. Esta esctrictamente prohibido utilizar variaciones que resuelvan directamente este problema, como `PyTorch Lightning` o `TensorFlow`. Los pasos a completar son los siguientes:

#### Cargar el dataset.

In [ ]:
!wget https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/new/assignment_1/train/train.tsv

In [ ]:
# Dataset.
dataset_df = pd.read_csv("https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/new/assignment_1/train/train.tsv", sep="\t")
codes, unique_labels = pd.factorize(dataset_df['clase'])
dataset_df['clase_id'] = codes + 1 #Se asocia un valor numérico a la clase, lo necesitamos para el DataLoader
dataset_df.head()

In [ ]:
target_classes

In [ ]:
from sklearn.model_selection import train_test_split

#Hago 2 splits, para poder obtener data de entrenamiento, validación y test
#train_validation_df: Data de entrenamiento junto a data de validacion
#test_df: data para test
#train_df: data de entrenamiento obtenida de la division de train_validation_df
#val_df: data de validacion

train_validation_df, test_df = train_test_split(
    dataset_df,
    test_size=0.2,
    random_state=42,
    stratify=dataset_df["clase_id"] # Optional for balanced splits
)

train_df, val_df = train_test_split(
    train_validation_df,
    test_size=0.2,
    random_state=42,
    stratify=train_validation_df["clase_id"] # Optional for balanced splits
)


In [ ]:
print(f"La data de entrenamiento tiene {len(train_df)} cantidad de datos")
print(f"La data de test tiene {len(test_df)} cantidad de datos")
print(f"La data de validacion tiene {len(val_df)} cantidad de datos")


In [ ]:
#Ejemplos de datos
print(train_df.head(5))
print(test_df.head(5))
print(val_df.head(5))

In [ ]:
#Función para convertir los dataframe anteriormente creados al nuevo dataset que necesitamos

def load_dataset(df: pd.DataFrame):
    for row in df.itertuples():
        yield int(row.clase_id), row.texto

In [ ]:
train_dataset = load_dataset(train_df)
test_dataset = load_dataset(test_df)

#### Definir el vocabulario.

In [ ]:
from torchtext.vocab import build_vocab_from_iterator

In [ ]:
import re

def get_tokens(texto):
  for word in texto:
        clean_text = re.sub(r'[^A-Za-zñáéíóú]', ' ', word)
        tokens = clean_text.split()
  return tokens

In [ ]:
def build_vocabulary(datasets):
    for dataset in datasets:
        for _, text in dataset:
            yield get_tokens(text)

vocab = build_vocab_from_iterator(build_vocabulary([train_dataset, test_dataset]), min_freq=1, specials=["<UNK>"])

vocab.set_default_index(vocab["<UNK>"])

#### Cargar el `DataLoader`

Recuerde que podría necesitar una función intermedia para procesar cada batch durante el entrenamiento, pero no es obligatorio hacerlo.

In [ ]:
train_dataset = load_dataset(train_df)
test_dataset = load_dataset(test_df)

In [ ]:
train_dataset_map_style, test_dataset_map_style = to_map_style_dataset(train_dataset), to_map_style_dataset(test_dataset)

max_words = 25

def vectorize_batch(batch):
    Y, X = list(zip(*batch))
    X = [vocab(get_tokens(text)) for text in X]
    X = [tokens+([0]* (max_words-len(tokens))) if len(tokens)<max_words else tokens[:max_words] for tokens in X]

    return torch.tensor(X, dtype=torch.int32), torch.tensor(Y) - 1

train_loader = DataLoader(train_dataset_map_style, batch_size=1024, collate_fn=vectorize_batch, shuffle=True)
test_loader  = DataLoader(test_dataset_map_style , batch_size=1024, collate_fn=vectorize_batch)

#### Definir la red recurrente.

Recuerde que debe difirnir los hyperparametros que estime conveniente.

In [ ]:
import torch
import torch.nn as nn

embed_len = 128
hidden_dim = 256
n_layers = 2
num_classes = len(dataset_df["clase"].unique())
dropout_rate = 0.2

# --- LSTM ---
class RNNClassifier(nn.Module):
    def __init__(self):
        super(RNNClassifier, self).__init__()

        # 1. Embedding Layer: Converts token IDs into dense vectors
        self.embedding_layer = nn.Embedding(num_embeddings=len(vocab), embedding_dim=embed_len)

        # 2. LSTM Layer: The core recurrent layer.
        # dropout is applied to the output of each layer except the last.
        self.lstm = nn.LSTM(
            input_size=embed_len,
            hidden_size=hidden_dim,
            num_layers=n_layers,
            batch_first=True,
            dropout=dropout_rate
        )

        self.linear = nn.Linear(hidden_dim, num_classes)

        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

    def forward(self, X_batch):
        embeddings = self.embedding_layer(X_batch)
        batch_size = X_batch.size(0)
        device = X_batch.device

        h0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
        c0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)

        output, (hn, cn) = self.lstm(embeddings, (h0, c0))

        return self.linear(hn[-1])

#### Funciones de entrenamiento y evaluación.

Para esta parte, puede utilizar las funciones vista en el tutorial directamente. Pero es su reponsabilidad ajustarlas a su código.

In [ ]:
def CalcValLossAndAccuracy(model, loss_fn, val_loader):
    with torch.no_grad():
        Y_shuffled, Y_preds, losses = [],[],[]
        for X, Y in val_loader:
            preds = model(X)
            loss = loss_fn(preds, Y)
            losses.append(loss.item())

            Y_shuffled.append(Y)
            Y_preds.append(preds.argmax(dim=-1))

        Y_shuffled = torch.cat(Y_shuffled)
        Y_preds = torch.cat(Y_preds)

        print("Valid Loss : {:.3f}".format(torch.tensor(losses).mean()))
        print("Valid Acc  : {:.3f}".format(accuracy_score(Y_shuffled.detach().numpy(), Y_preds.detach().numpy())))


def TrainModel(model, loss_fn, optimizer, train_loader, val_loader, epochs=10):
    for i in range(1, epochs+1):
        losses = []
        for X, Y in tqdm(train_loader):
            Y_preds = model(X)

            loss = loss_fn(Y_preds, Y)
            losses.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print("Train Loss : {:.3f}".format(torch.tensor(losses).mean()))
        CalcValLossAndAccuracy(model, loss_fn, val_loader)

def MakePredictions(model, loader):
    Y_shuffled, Y_preds = [], []
    for X, Y in loader:
        preds = model(X)
        Y_preds.append(preds)
        Y_shuffled.append(Y)
    gc.collect()
    Y_preds, Y_shuffled = torch.cat(Y_preds), torch.cat(Y_shuffled)

    return Y_shuffled.detach().numpy(), F.softmax(Y_preds, dim=-1).argmax(dim=-1).detach().numpy()



##### Entrenamiento del modelo

Ejecute el entrenamiento de su modelo propuesto.

In [ ]:
epochs = 5
learning_rate = 1e-3

loss_fn = nn.CrossEntropyLoss()
rnn_classifier = RNNClassifier()
optimizer = Adam(rnn_classifier.parameters(), lr=learning_rate)

TrainModel(rnn_classifier, loss_fn, optimizer, train_loader, test_loader, epochs)

##### Evaluacion del modelo

Ejecute la evaluación de su modelo, y genere un reporte de evaluación similar al de la pregunta anterior.

In [ ]:
target_classes = ["normal", "incivilidad", "odio"] #[1,2,3]

In [ ]:
Y_actual, Y_preds = MakePredictions(rnn_classifier, test_loader)

In [ ]:
print("Test Accuracy : {}".format(accuracy_score(Y_actual, Y_preds)))
print("\nClassification Report : ")
print(classification_report(Y_actual, Y_preds, target_names=target_classes))
print("\nConfusion Matrix : ")
print(confusion_matrix(Y_actual, Y_preds))

**Finalmente, análice sus resultados, ¿Porqué cree que obtuvo esos resultados?, ¿Es mejor que sólo utilizar Word Embeddings, porque?. Justique.**

Dado que escogimos utilizar Long Short-Term Memory (LSTM), respondemos esta parte considerando lo anterior. Al utilizar Word Embeddings con LSTM en general es mejor que utilizar solamente Word Embeddings; al usar sólo Word Embeddings, no toma el orden secuencial ni tampoco el contexto de las palabras dentro de una oración; al usar Word Embeddings con LSTM, resuelve la limitación secuencial y entendiendo además cómo las palabras al principio de una oración influyen en las palabras del final.

### Transformers BERT.

Para esta tarea se le piden crear una representación de texto usando la arquitectura basada en transformers, BERT:



In [ ]:
!pip install transformers


#### Import BETO

Para esto debe importar el tokenizador y el modelo BETO.

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

#### Ejemplo de extracción de features.

In [ ]:
# Uninstall the older version and then install the required modern version
!pip uninstall torch -y
!pip install --upgrade torch torchvision torchaudio

Luego, debe cargar los modelos pre-entrenados:

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')
model = AutoModelForMaskedLM.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased', output_hidden_states=True).to(device)

Una vez cargado BETO, debe utilizarlo para extraer los embeddings asociados a la texto de su corpus. Se espera que usted realice lo siguiente:



Tokenizamos el texto para extraer los ids a cada palabra en el vocabulario interno de BETO, se recomienda utilizar el padding, trunctation, y max_length para considerar oraciones de diferentes tamaños.

Luego, debe verificar si cada uno de los ids extraídos se encuentran en la misma máquina donde fue cargado el modelo (CPU o GPU), se recomienda dejar todo en GPU.

In [ ]:
# oración
sentence = "Hola, que tal? me gusta mucho el curso de NLP"

# extraemos los ids de los tokens, se recomienda definir los valores de las variables:
#  padding, truncation, max_length debido a que las secuencia de texto puede tener diferentes largos
inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt", max_length=512)
# revisamos si cada de los ids, se encuentran en la misma máquina que el modelo (GPU o CPU)
inputs = {k: v.to(model.device) for k, v in inputs.items()}
inputs

Una vez, extraídos los ids, usted debe obtener los estados ocultos de las ultimas capas de BETO.

In [ ]:
# Extraemos la features
outputs = model(**inputs)

# ahora `outputs` tendrá el atributo `hidden_states`
hidden_states = outputs.hidden_states


Finalmente, debe guardar los embeddings en CPU los embeddings del token [CLS] que será usados en la clasificación del análisis de sentimientos.

In [ ]:
import torch

In [ ]:
with torch.no_grad():
# Seleccionamos la última capa y obtenemos el primer token ([CLS]) para cada ejemplo
# Estos son los embeddings que normalmente se usan para tareas de clasificación.
  cls_embeddings = hidden_states[-1][:, 0, :].cpu().numpy()


#### Extraer features de todo el dataset

Considerando lo anterior, usted debe implementar la función `get_beto_features_in_batches` para extraer los features de BETO (los estados ocultos y los embeddings del token [CLS]).

Esta función recibe dos parámetros, el texto a vectorizar y un tamaño de batch, debido a que es extramadamente recomendable a que procesen el texto por lotes, ya que si cargan todos el modelo se les agotará la memoria RAM.

In [ ]:
#Si es necesario, cargamos la data nuevamente

import pandas as pd

!wget https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/new/assignment_1/train/train.tsv

dataset_df = pd.read_csv("https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/new/assignment_1/train/train.tsv", sep="\t")
codes, unique_labels = pd.factorize(dataset_df['clase'])
dataset_df['clase_id'] = codes + 1 #Se asocia un valor numérico a la clase, lo necesitamos para el DataLoader
dataset_df.head()

from sklearn.model_selection import train_test_split

train_validation_df, test_df = train_test_split(
    dataset_df,
    test_size=0.2,
    random_state=42,
    stratify=dataset_df["clase_id"] # Optional for balanced splits
)

train_df, val_df = train_test_split(
    train_validation_df,
    test_size=0.2,
    random_state=42,
    stratify=train_validation_df["clase_id"] # Optional for balanced splits
)


In [ ]:
from datasets import load_dataset, Dataset, DatasetDict

In [ ]:
#Se crea un DatasetDict con la data de entrenamient, test y validación

train_dataset = Dataset.from_dict({
    'text': train_df['texto'].tolist(),
    'label': train_df['clase_id'].tolist()
})

validation_dataset = Dataset.from_dict({
    'text': val_df['texto'].tolist(),
    'label': val_df['clase_id'].tolist()
})

test_dataset = Dataset.from_dict({
    'text': test_df['texto'].tolist(),
    'label': test_df['clase_id'].tolist()
})

my_dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset,
    'test': test_dataset,
})

In [ ]:
my_dataset_dict

In [ ]:
def tokenize(batch):
  # Por cada batch vamos a obtener el texto, tokenizarlo y aplicar padding
  return tokenizer(batch["text"], padding=True, truncation=True, max_length=256)

def extract_hidden_states(batch):
 # Vamos a extraer los hidden states desde BERT, para esto pasamos el modelo
 # dispositivo
 inputs = {k:v.to(device) for k,v in batch.items() if k in tokenizer.model_input_names}
 # Extraemos solamente la last hidden state
 with torch.no_grad():
   # Changed model_distilbert to model_beto
   # Corrected: Access hidden_states tuple and take the last element
   last_hidden_state = model(**inputs).hidden_states[-1]
 # representaciones vectoriales que tanto queriamos :D
 return {"hidden_state": last_hidden_state[:,0].cpu().numpy()}

In [ ]:
# Función para procesar los textos en lotes y obtener las características de BETO

BATCH_SIZE = 16

def get_beto_features_in_batches(texts, batch_size):
  texts_encoded = texts.map(tokenize, batched=True, batch_size=BATCH_SIZE)
  texts_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label", "text"])
  texts_hidden = texts_encoded.map(extract_hidden_states, batched=True, batch_size=BATCH_SIZE)
  return texts_hidden

device


Ahora extraíga los features, un punto importante es que la extracción de features podría tomar alrededor de una a dos horas dependiendo del tamaño del batch que utilicen.

In [ ]:
train_embs = get_beto_features_in_batches(my_dataset_dict, BATCH_SIZE)

In [ ]:
train_embs

#### Clasificación

Una vez extraído los features de BETO, realice la clasificación de los embeddings obtenidos. Debe implementar dos clasificadores a su elección.

In [ ]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# Se extraen las features obtenidas con get_beto_features_in_batches para entrenamiento y test

X_train = np.array(train_embs['train']['hidden_state'])
y_train = np.array(train_embs['train']['label'])

X_test = np.array(train_embs['test']['hidden_state'])
y_test = np.array(train_embs['test']['label'])

print(f"Train Features Shape: {X_train.shape}")
print(f"Test Features Shape: {X_test.shape}")

In [ ]:
#Se crean los 2 clasificadores

# Clasificador 1
clf1 = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(random_state=42, n_jobs=-1, n_estimators=100))
])

# Clasificador 2
clf2 = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(
      C=1.0,
      max_iter=1000,
      solver='liblinear',
      random_state=42
  ))
])

In [ ]:
clf1.fit(X_train, y_train)


In [ ]:
clf2.fit(X_train, y_train)

#### Reporte de evaluación

Una vez realizada la clasificación, realice el reporte de clasificación y el análsis de la matriz confusión para ambos clasificadores.

Recuerde que para hacer esto, debe extraer los features del dataset de testing.

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
#Reporte clasificador 1
y_pred = clf1.predict(X_test)
report = classification_report(y_test, y_pred)

print("\n--- Classification Report (Test Set) ---")
print(report)

In [ ]:
#Reporte clasificador 2
y_pred = clf2.predict(X_test)
report = classification_report(y_test, y_pred)

print("\n--- Classification Report (Test Set) ---")
print(report)

**Finalmente, que puede decir de los resultados obtenidos. ¿Se diferencia de los resultados obtenidos de la red RNN? ¿A que se debe esto? Justifique**


Teóricamente, BERT (Bidirectional Encoder Representations from Transformers) es mejor y más potente que usar una arquitectura basada en LSTM con Word Embeddings, como la utilizada anteriormente. Por ejemplo, en el primer modelo obtuvimos un macro avg de la precision y recall de 0.15 y 0.33, mientras que con BERT/BETO con el 1er clasificador obtuvimos 0.71 y 0.56, y con el segundo 0.65 y 0.63 respectivamente.
BERT es bidireccional (con su arquitectura de Transformer Encoder), mirando simultáneamente las palabras que preceden y suceden a un token en particular (Self-Attention).
BERT maneja un significado dinámico, como el ejemplo de banco (entidad financiera vs un asiento).
Etc.

### Large Language Model




##### Zero Shot Learning

Utilizando la técnica de zero shot learning, utilice la API de `openai` para clasificar el texto del dataset.

Además, genere el reporte de clasificación usando `scikit-learn` como en las preguntas anteriores.

Recuerde solicitar al profesor auxiliar el TOKEN para hacer consultas al LLM.

In [ ]:
import requests
from openai import OpenAI
from pydantic import BaseModel
import json
from sklearn.metrics import classification_report

In [ ]:
def openAiUso(prompt):
    API_KEY = "sk-xePE_4Bi13W2LnR--EG83Q"
    url = "https://models.villena.cl/v1/responses"

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {API_KEY}"
    }

    data = {
        "model": "openai/gpt-5-nano",
        "input": prompt,
    }

    response = requests.post(url, headers=headers, json=data)
    data = response.json()
    return data["output"][1]["content"][0]["text"]


In [ ]:
#Se realiza una muestra solo de los primeros 20 tweets debido al tiempo de respuesta por consulta a openAi
y_test = test_df["clase"].tolist()[:20]
X_test_tweets = test_df["texto"].tolist()[:20]
print(X_test_tweets)

In [ ]:
print(test_df["clase"])

In [ ]:
print(len(X_test_tweets))

In [ ]:
print(len(dataset_df))

In [ ]:
tweets = X_test_tweets
prompts = []
instruccion="Eres un experto en clasificación de textos en redes sociales. Tu tarea es leer un tweet en español y asignarlo a una y solo una de las siguientes categorías: odio, incivilidad o normal. Odio: cualquier expresión que promueva o incite a la discriminación, la hostilidad o la violencia hacia una persona o grupo en una relación asimétrica de poder, por razones como raza, etnia, género, orientación sexual, religión, nacionalidad, discapacidad u otra característica similar. Incivilidad: cualquier comportamiento o actitud que rompa las normas de respeto, cortesía o consideración entre personas, incluyendo insultos, ataques personales, sarcasmo, desprecio u otras formas de agresión verbal que no constituyen discurso de odio. Normal: expresiones que no contienen hostilidad, ataques, incivilidad ni discurso de odio. Debes responder solo con una de estas tres palabras: odio, incivilidad, normal. No entregues explicaciones ni texto adicional. Este es el twets: "
for tweet in tweets:
  prompts.append(instruccion+tweet)


In [ ]:
y_pred = []
##print(len(prompts))

for prompt in prompts:
  prediccion=openAiUso(prompt)
  limpio = prediccion.strip().lower()
  y_pred.append(limpio)


In [ ]:
report = classification_report(y_test, y_pred, zero_division=0)
print(report)

##### Few Shot Learning

Utilizando la técnica de few shot learning, utilice la API de `openai` para clasificar el texto del dataset.

Además, genere el reporte de clasificación usando `scikit-learn` como en las preguntas anteriores.

Recuerde solicitar al profesor auxiliar el TOKEN para hacer consultas al LLM.

In [ ]:
tweets = X_test_tweets
promptsFewShotLearning = []
instruccion="Eres un experto en clasificación de textos en redes sociales. Tu tarea es leer un tweet en español y asignarlo a una y solo una de las siguientes categorías: odio, incivilidad o normal. Odio: cualquier expresión que promueva o incite a la discriminación, la hostilidad o la violencia hacia una persona o grupo en una relación asimétrica de poder, por razones como raza, etnia, género, orientación sexual, religión, nacionalidad, discapacidad u otra característica similar. Incivilidad: cualquier comportamiento o actitud que rompa las normas de respeto, cortesía o consideración entre personas, incluyendo insultos, ataques personales, sarcasmo, desprecio u otras formas de agresión verbal que no constituyen discurso de odio. Normal: expresiones que no contienen hostilidad, ataques, incivilidad ni discurso de odio. Debes responder solo con una de estas tres palabras: odio, incivilidad, normal. No entregues explicaciones ni texto adicional. Ejemplos de estos serian. La persona que me atendio tiene un problemas de actitud. Clasificación: incivilidad. La forma en que actuo estuvo bien. Los migrantes son una plaga que debería ser expulsada. Clasificación: odio . Eres un imbécil, no sabes nada. Clasificación: incivilidad. Hoy cociné por primera vez pastel de choclo y quedó buenísimo. Clasificación: normal. ¿Alguien sabe si mañana llueve en Santiago?. Clasificación: normal. Gracias a todos por el apoyo estos días, se valora mucho. Clasificación:normal. Este es el twets: "
for tweet in tweets:
  promptsFewShotLearning.append(instruccion+tweet)

In [ ]:
y_pred = []
##print(len(prompts))

for prompt in promptsFewShotLearning:
  prediccion=openAiUso(prompt)
  limpio = prediccion.strip().lower()
  y_pred.append(limpio)

In [ ]:
reportFewShotLearning = classification_report(y_test, y_pred, zero_division=0)
print(reportFewShotLearning)

##Comparación

Dado los resultados de los dos Large Language Model, el modelo Few Shot Learning obtiene un mejor rendimiento, al mostrar una mayor accuracy (0.75 vs 0.70) y mejores valores tanto en macro F1 (0.75 vs 0.70) como en weighted F1 (0.74 vs 0.69). En resumen, esto indica que clasifica de forma mejor que el modelo Zero Shot.